In [28]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree



In [29]:
data = pd.read_csv('features.csv')
# Delete unesessary features from dataset
data = data.dropna()
data = data.drop(['enterworld_num','buyitemnowmainauction_num','completechallengeweek_num'],axis=1)
data = data.drop(['actor_account_id'
                    ,'survival_time'
                    ],axis=1)
data = data.drop(columns=data.columns[0],axis=1)
# Setup & split dataset for training
X = data.copy()
y = X.pop('churn_yn')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)
# Print training data shape
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)
# Get feature names from training set
feature_names = [f"feature {i}" for i in range(X.shape[1])]

(995, 33) (995,)
(2985, 33) (2985,)


In [30]:
clf = DecisionTreeClassifier(criterion='entropy', 
    splitter='best', 
    max_depth=6, 
    min_samples_split=2, 
    min_samples_leaf=4, 
    min_weight_fraction_leaf=0.0, 
    max_features='sqrt', 
    random_state=None, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    ccp_alpha=0.0,
    class_weight={0:1, 1:1 , 2:0.9 , 3:0.2, 4:0.2, 5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0}
)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [31]:
forest = RandomForestClassifier(random_state=0,
                                criterion='entropy',
                                max_depth=6,
                                max_features='sqrt',
                                n_estimators=1000)
forest.fit(X_train, y_train)
predictions = forest.predict(X_test)

In [32]:
svc = SVC(probability=True)


In [33]:
X = data.copy()
y = X.pop('churn_yn')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)

In [35]:
eclf = VotingClassifier(estimators=[('RF', forest),
                                    ('SVC', svc),
                                    ('DCTree', clf)],
                                    voting='soft', weights=[2,1,2])

eclf.fit(X_train, y_train)

VotingClassifier(estimators=[('RF',
                              RandomForestClassifier(criterion='entropy',
                                                     max_depth=6,
                                                     n_estimators=1000,
                                                     random_state=0)),
                             ('SVC', SVC(probability=True)),
                             ('DCTree',
                              DecisionTreeClassifier(class_weight={0: 1, 1: 1,
                                                                   2: 0.9,
                                                                   3: 0.2,
                                                                   4: 0.2, 5: 0,
                                                                   6: 0, 7: 0,
                                                                   8: 0, 9: 0,
                                                                   10: 0, 11: 0,
                                                                   12: 0, 13: 0,
                                                                   14: 0, 15: 0,
                                                                   16: 0, 17: 0,
                                                                   18: 0,
                                                                   19: 0},
                                                     criterion='entropy',
                                                     max_depth=6,
                                                     max_features='sqrt',
                                                     min_samples_leaf=4))],
                 voting='soft', weights=[2, 1, 2])

In [36]:
filename = 'voting_clf.sav'
pickle.dump(eclf, open(filename, 'wb'))